In [1]:
# Disable TensorFlow debugging info and warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 2: Info and warnings not displayed 

In [2]:
import numpy as np
import skimage.io
import matplotlib.pyplot as plt
import pathlib
import tqdm
import annotation, misc, hyspec_io, image_render, hyspec_ml, preprocess
import skimage.exposure
import tensorflow as tf

In [3]:
# Disable GPUs
tf.config.set_visible_devices([], 'GPU')
visible_devices = tf.config.get_visible_devices()
for device in visible_devices:
    assert device.device_type != 'GPU'

In [4]:
# Paths
pca_data_model_file = pathlib.Path('/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/M_PCA_Model/20210825_OlbergAreaA_PCA_data_and_model.npz')
hyspec_dir = pathlib.Path('/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/4a_Rad_Georef_SGC')
labels_dir = pathlib.Path('/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/M_Annotation/20210825_Olbergholmen_Annotation_20230228_gray/OlbergAreaS')
pca_im_dir = pathlib.Path('/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/5a_Rad_Georef_SGC_PCA')

In [5]:
# Load PCA model
with np.load(pca_data_model_file) as npz_files:
    X_mean = npz_files['spectra_mean']
    X_std = npz_files['spectra_std']
    W_pca = npz_files['pca_components']

In [6]:
# Get filename paths
label_image_paths = sorted([file for file in labels_dir.glob('*.png')])
hyspec_image_paths = [hyspec_dir / (file.stem + '.bip.hdr') for file in label_image_paths]

In [7]:
for hyspec_image_path, label_image_path in tqdm.tqdm(zip(hyspec_image_paths, label_image_paths),total=len(hyspec_image_paths)):
    # Load hyperspectral image
    image,wl,rgb_ind,_ = hyspec_io.load_envi_image(str(hyspec_image_path))

    # Load label image
    label_image = skimage.io.imread(label_image_path)
    
    # Create PCA image
    im_pca = hyspec_ml.pca_transform_image(image,W_pca,X_mean,X_std)
    
    # Inpaint PCA image
    im_pca_inpaint = preprocess.inpaint_missing_pixels(im_pca)
    
    # Set labels for zero-valued pixels to background
    label_image[np.all(im_pca_inpaint,axis=2)==0] = 0
    
    # Save as tensorflow dataset
    dataset = tf.data.Dataset.from_tensors((im_pca_inpaint,label_image))  # Use tf.data.Dataset.concatenate to merge later
    dataset_save_path = pca_im_dir / label_image_path.stem
    tf.data.experimental.save(dataset,str(dataset_save_path))
    

  0%|                                                    | 0/18 [00:00<?, ?it/s]

  6%|██▍                                         | 1/18 [00:06<01:57,  6.91s/it]

 11%|████▉                                       | 2/18 [01:56<18:01, 67.59s/it]

 17%|███████▎                                    | 3/18 [02:37<13:48, 55.25s/it]

 22%|█████████▊                                  | 4/18 [03:09<10:47, 46.23s/it]

...

 78%|█████████████████████████████████▍         | 14/18 [20:59<04:59, 74.77s/it]

 83%|███████████████████████████████████▊       | 15/18 [21:15<02:51, 57.03s/it]

 89%|██████████████████████████████████████▏    | 16/18 [21:22<01:23, 41.89s/it]

 94%|████████████████████████████████████████▌  | 17/18 [21:59<00:40, 40.61s/it]

100%|███████████████████████████████████████████| 18/18 [24:14<00:00, 80.82s/it]
